# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [29]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [30]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [31]:
# frequency table for prestige and whether or not someone was admitted

pd.crosstab(df['prestige'], df['admit'])

admit,0,1
prestige,,
1.0,28,33
2.0,95,53
3.0,93,28
4.0,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [32]:
df = pd.get_dummies(df, columns=['prestige'])

#### 2.2 When modeling our class variables, how many do we need? 



    Answer: Three. When creating dummies, we need the number of total categories - 1. If there is a null value for all of our dummy variables, we know the instance had a value of the category that we didn't create a dummy variable out of. But for now we will keep all four that this creates without drop_first

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [33]:
df.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [34]:
cols_to_keep = ['admit', 'gre', 'gpa']
dummy_ranks = df
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1.0':])
print handCalc.head()

#i just adapted the code that was here before, but don't really understand why it needed to be here.

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


/Users/bjr-fov/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

crosstabCalc = pd.crosstab(handCalc['prestige_1.0'], handCalc['admit'])
print crosstabCalc

admit           0   1
prestige_1.0         
0             243  93
1              28  33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [36]:
odds_number1 = (float (crosstabCalc[1][1])/((crosstabCalc[0][1]) + (crosstabCalc[1][1])))

print odds_number1

0.540983606557


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [37]:
odds_notnumber1 = (float (crosstabCalc[1][0])/((crosstabCalc[1][0]) + crosstabCalc[0][0]))

print odds_notnumber1

0.276785714286


#### 3.3 Calculate the odds ratio

In [38]:
print odds_number1/odds_notnumber1

1.95452141724


#### 3.4 Write this finding in a sentenance: 

Answer: If you go to a #1 rank school you are almost twice as likely to be admitted into the program than if you did not go to a #1 ranked school.

#### 3.5 Print the cross tab for prestige_4

In [40]:
crosstabCalc = pd.crosstab(handCalc['prestige_4.0'], handCalc['admit'])


#### 3.6 Calculate the OR 

In [41]:
odds_number4 = (float (crosstabCalc[1][1])/((crosstabCalc[0][1]) + (crosstabCalc[1][1])))
odds_notnumber4 = (float (crosstabCalc[1][0])/((crosstabCalc[1][0]) + crosstabCalc[0][0]))
print odds_number4/odds_notnumber4


0.518460329929


#### 3.7 Write this finding in a sentence

Answer: An applicant is half as likely to be admitted if they go to a bottom-ranked school than if they went to a non-bottom-ranked school. 

## Part 4. Analysis

In [42]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [43]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [44]:
train_cols = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

#### 4.2 Fit the model

In [48]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()
print result.summary()

Optimization terminated successfully.
         Current function value: 0.589121
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Mon, 28 Aug 2017   Pseudo R-squ.:                 0.05722
Time:                        07:55:06   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0014      0.001      1.308      0.191      -0.001       0.003
gpa             -0.1323

#### 4.3 Print the summary results

In [49]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      392
Method:                           MLE   Df Model:                            4
Date:                Mon, 28 Aug 2017   Pseudo R-squ.:                 0.05722
Time:                        07:55:08   Log-Likelihood:                -233.88
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.039e-05
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0014      0.001      1.308      0.191      -0.001       0.003
gpa             -0.1323      0.195     -0.680      0.497      -0.514       0.249
prestige_2.0    -0.9562      0.302     -3.17

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [50]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           0.999320  1.003420  1.001368
gpa           0.598303  1.282800  0.876073
prestige_2.0  0.212826  0.694082  0.384342
prestige_3.0  0.112055  0.412207  0.214918
prestige_4.0  0.070176  0.338540  0.154135


#### 4.5 Interpret the OR of Prestige_2

If you go to a second rated school, your chance of getting into the program is about 40% of if you went to a school with the other prestige levels.

#### 4.6 Interpret the OR of GPA

With every increase in 1 in GPA, your likelihood of being accepted increases by 100%

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [51]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [23]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

print combos

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]
         0         1    2    3
0    220.0  2.260000  1.0  1.0
1    220.0  2.260000  2.0  1.0
2    220.0  2.260000  3.0  1.0
3    220.0  2.260000  4.0  1.0
4    220.0  2.453333  1.0  1.0
5    220.0  2.453333  2.0  1.0
6    220.0  2.453333  3.0  1.0
7    220.0  2.453333  4.0  1.0
8    220.0  2.646667  1.0  1.0
9    220.0  2.646667  2.0  1.0
10   220.0  2.646667  3.0  1.0
11   220.0  2.646667  4.0  1.0
12   220.0  2.840000  1.0  1.0
13   220.0  2.840000  2.0  1.0
14   220.0  2.840000  3.0  1.0
15   220.0  2.840000  4.0  1.0
16   220.0  3.033333  1.0  1.0
17   220.0  3.033333  2.0  1.0
18   220.0  3.033333  3.0  1.0
19   220.0  3.033333  4.0  1.0
20   220.0  3.226667  1.0  1.0
21   220.0  3.226667  2.0  1.0
22   220.0  3.226667 

#### 5.1 Recreate the dummy variables

In [57]:
# recreate the dummy variables

# keep only what we need for making predictions

#combos = pd.get_dummies(combos, columns=[2], drop_first = True)

print combos

         0         1    3  2_2.0  2_3.0  2_4.0
0    220.0  2.260000  1.0      0      0      0
1    220.0  2.260000  1.0      1      0      0
2    220.0  2.260000  1.0      0      1      0
3    220.0  2.260000  1.0      0      0      1
4    220.0  2.453333  1.0      0      0      0
5    220.0  2.453333  1.0      1      0      0
6    220.0  2.453333  1.0      0      1      0
7    220.0  2.453333  1.0      0      0      1
8    220.0  2.646667  1.0      0      0      0
9    220.0  2.646667  1.0      1      0      0
10   220.0  2.646667  1.0      0      1      0
11   220.0  2.646667  1.0      0      0      1
12   220.0  2.840000  1.0      0      0      0
13   220.0  2.840000  1.0      1      0      0
14   220.0  2.840000  1.0      0      1      0
15   220.0  2.840000  1.0      0      0      1
16   220.0  3.033333  1.0      0      0      0
17   220.0  3.033333  1.0      1      0      0
18   220.0  3.033333  1.0      0      1      0
19   220.0  3.033333  1.0      0      0      1
20   220.0  3

In [58]:
combos.columns = ['gre', 'gpa', 'intercept', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

#### 5.2 Make predictions on the enumerated dataset

In [60]:
combos['admit_pred'] = result.predict(combos[train_cols])
print combos

       gre       gpa  intercept  prestige_2.0  prestige_3.0  prestige_4.0  \
0    220.0  2.260000        1.0             0             0             0   
1    220.0  2.260000        1.0             1             0             0   
2    220.0  2.260000        1.0             0             1             0   
3    220.0  2.260000        1.0             0             0             1   
4    220.0  2.453333        1.0             0             0             0   
5    220.0  2.453333        1.0             1             0             0   
6    220.0  2.453333        1.0             0             1             0   
7    220.0  2.453333        1.0             0             0             1   
8    220.0  2.646667        1.0             0             0             0   
9    220.0  2.646667        1.0             1             0             0   
10   220.0  2.646667        1.0             0             1             0   
11   220.0  2.646667        1.0             0             0             1   

#### 5.3 Interpret findings for the last 4 observations

Answer: The scores in the fourth to last (with high scores and a prestige of 1), the chances of getting in are about 64%; with a prestige of 2, a chance of 40%; with a prestige of 3, 27%; with a prestige of 4, 21%.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.